# Module 1 - Final Project

In [69]:
# Imports
import requests
import pandas as pd
from bs4 import BeautifulSoup
import re
import gspread
from oauth2client.service_account import ServiceAccountCredentials
import tweepy as tw
import json

In [70]:
# Base table scraped
# Retrieve table of European metal festivals of 2019
url = 'https://www.festival-alarm.com/us/Categories/Metal-festivals/(year)/2019'

try:
    html = requests.get(url).content
except requests.exceptions.HTTPError as errh:
    print ("Http Error:",errh)
except requests.exceptions.ConnectionError as errc:
    print ("Error Connecting:",errc)
except requests.exceptions.Timeout as errt:
    print ("Timeout Error:",errt)
except requests.exceptions.RequestException as err:
    print ("OOps: Something Else",err)

soup = BeautifulSoup(html, 'html')
fest_table = soup.find('table', {'class': 'festival_table'})
#rows = [row.text.strip().split('\n') for row in fest_table.find_all('tr')]
colnames = ['Name', 'Dates', 'Duration', 'Environment', 'Genres', 'Country', 'Venue', 'Price']
#data = rows[1:]
# Columns data
fest_names = [name.text.strip() for name in fest_table.find_all('td', {'class': 'event-title'})]
genres = [genre.text.strip() for genre in fest_table.find_all('td', {'class': 'event-genre'})]
countries = [country.text.strip() for country in fest_table.find_all('td', {'class': 'event_country'})]
venues = [''.join(venue.text.strip().split('\n')) for venue in fest_table.find_all('td', {'class': 'event-venue'})]
prices = [float(price.text.strip().replace('€ ', '').replace(',', '').replace('n/a', '0.00')) for price in fest_table.find_all('td', {'class': 'ticket_price'})]
durations = [''.join(re.findall(r'\d{1} day', duration.text)) for duration in fest_table.find_all('tr', {'class': 'event-wrapper'})]
dates = [' - '.join(re.findall(r'\d{2}/\d{2}', date.text)) for date in fest_table.find_all('tr', {'class': 'event-wrapper'})]
envs = [''.join(re.findall(r'indoor|outdoor', env.text)) for env in fest_table.find_all('tr', {'class': 'event-wrapper'})]
#visitors = [re.findall(r'^\d+|no data', visitor.text) for visitor in fest_table.find_all('tr', {'class': 'event-wrapper'})]


In [71]:
# Create the base DataFrame
df = pd.DataFrame(columns=colnames)
df['Name'] = fest_names
df['Dates'] = dates
df['Duration'] = durations
df['Environment'] = envs
df['Genres'] = genres
df['Country'] = countries
df['Venue'] = venues
df['Price'] = prices
df = df[df['Price'].between(90.00, 340.00)].sort_values(by='Price', ascending=False).set_index('Name')
df

,Dates,Duration,Environment,Genres,Country,Venue,Price
Name,,,,,,,
Copenhell,06/19 - 06/22,4 day,outdoor,"Folk, Metal, Rock",Denmark,"Refshalevej 185 København K, ...",340.00
Sweden Rock Festival,06/05 - 06/08,4 day,outdoor,"Hardcore (Metal), Metal, Medieval, Punk, Rock",Sweden,"Sölvesborg, SE29431",286.00
Wacken Open Air,08/01 - 08/03,3 day,outdoor,"Comedy, German rock, Folk, Gothic, Hardcore (M...",Schleswig-Holstein,"Wacken, DE25596",221.00
Graspop Metal Meeting,06/21 - 06/23,3 day,outdoor,"Gothic, Hardcore (Metal), Metal, Punk, Rock",Belgium,"Dessel, BE2480",210.00
Inferno Festival,04/18 - 04/21,4 day,indoor,"Hardcore (Metal), Metal, Punk, Rock",Norway,"Rockefeller, John Dee, Kniven & Vaterland Oslo...",199.00
Bloodstock Open Air,08/08 - 08/11,4 day,outdoor,"Metal, Rock",United Kingdom,"Catton Park Walton-On-Trent, ...",176.00
Metaldays Festival,07/21 - 07/27,7 day,outdoor,"Metal, Rock",Slovenia,"Tolmin, SI5216",175.00
Download Festival (GB),06/14 - 06/16,3 day,outdoor,"Metal, Rock",United Kingdom,"Donington Park Derby, ...",164.00
Sabaton Open Air,08/14 - 08/17,4 day,outdoor,"Metal, Rock",Sweden,"Falun, SE79130",160.00


In [116]:
# Google Sheet with lineups from selected festivals
# use creds to create a client to interact with the Google Drive API
scope = ['https://spreadsheets.google.com/feeds',
'https://www.googleapis.com/auth/drive']
creds = ServiceAccountCredentials.from_json_keyfile_name('client_secret.json', scope)
client = gspread.authorize(creds)

sheet = client.open("2019_metal_fests_lineup").sheet1

# Extract and print all of the values
festivals = sheet.get_all_records()
lineups = [[fest['lineup']] for fest in list_of_hashes]

lineups_dict = {fest['festival']: fest['lineup'] for fest in list_of_hashes}
df['Bands'] = lineups_dict.values()
df

,Dates,Duration,Environment,Genres,Country,Venue,Price,Bands
Name,,,,,,,,
Copenhell,06/19 - 06/22,4 day,outdoor,"Folk, Metal, Rock",Denmark,"Refshalevej 185 København K, ...",340.00,"Tool,Slipknot,Scorpions,Slash featuring Myles ..."
Sweden Rock Festival,06/05 - 06/08,4 day,outdoor,"Hardcore (Metal), Metal, Medieval, Punk, Rock",Sweden,"Sölvesborg, SE29431",286.00,"A.C.T,Amon Amarth,Unleashed,Arch Enemy,At the ..."
Wacken Open Air,08/01 - 08/03,3 day,outdoor,"Comedy, German rock, Folk, Gothic, Hardcore (M...",Schleswig-Holstein,"Wacken, DE25596",221.00,"Airbourne,All Hail The Yeti,Avatar,Critical Me..."
Graspop Metal Meeting,06/21 - 06/23,3 day,outdoor,"Gothic, Hardcore (Metal), Metal, Punk, Rock",Belgium,"Dessel, BE2480",210.00,"Agnostic Front,Amon Amarth,Anthrax,Bleed From ..."
Inferno Festival,04/18 - 04/21,4 day,indoor,"Hardcore (Metal), Metal, Punk, Rock",Norway,"Rockefeller, John Dee, Kniven & Vaterland Oslo...",199.00,"Taake,1349,Vomitory,Gaahls Wyrd,Hypocrisy,Cara..."
Bloodstock Open Air,08/08 - 08/11,4 day,outdoor,"Metal, Rock",United Kingdom,"Catton Park Walton-On-Trent, ...",176.00,"Rotting Christ,Ten Ton Slug,Footprints in the ..."
Metaldays Festival,07/21 - 07/27,7 day,outdoor,"Metal, Rock",Slovenia,"Tolmin, SI5216",175.00,"Dimmu Borgir,Tarja,Demons & Wizards,Hypocrisy,..."
Download Festival (GB),06/14 - 06/16,3 day,outdoor,"Metal, Rock",United Kingdom,"Donington Park Derby, ...",164.00,"Def Leppard,Slash,Whitesnake,Clutch,Blackberry..."
Sabaton Open Air,08/14 - 08/17,4 day,outdoor,"Metal, Rock",Sweden,"Falun, SE79130",160.00,"Kardinal Sin,Rave The Reqviem,PreHistoric Anim..."


In [73]:
# Twitter API
# Count the number of tweets with the festival hashtag

# Utility JSON function
def getJSONValue(val):
    with open('twitter_secret.json') as file:
        data = json.load(file)
        return data[val]

# Authentification
consumer_key = getJSONValue('API_KEY')
consumer_secret = getJSONValue('API_SECRET')
access_token = getJSONValue('ACCESS_TOKEN')
access_token_secret = getJSONValue('ACCESS_TOKEN_SECRET')

auth = tw.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tw.API(auth, wait_on_rate_limit=True)

hashtags = ['#' + name.lower().split('(')[0].strip().replace(' ', '') + '2019' for name in df.index.tolist()]
date_since = "2018-07-01"

def getTweetsCount(hashtag):
    tweets = tw.Cursor(api.search,
                  q=hashtag,     
                  lang="en",
                  since=date_since).items()
    return len(list(tweets))

trend_counts = [getTweetsCount(name) for name in hashtags]

In [ ]:
# Scraping from Twitter search page directly
from twitterscraper import query_tweets

trend_counts = [len(query_tweets(f'{hashtag}', 10)) for hashtag in hashtags]
trend_counts